In [1]:
%pip install --upgrade setuptools wheel 

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install wheel pymysql mysql-connector-python sqlalchemy schedule

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install git+https://github.com/heebp/pykrx.git


  Cloning https://github.com/heebp/pykrx.git to c:\users\student\appdata\local\temp\pip-req-build-72faip1y
  Resolved https://github.com/heebp/pykrx.git to commit b0ffe3651dfb7037d5494d03b16a04622aec5a6a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/heebp/pykrx.git 'C:\Users\student\AppData\Local\Temp\pip-req-build-72faip1y'


In [1]:
from pykrx import stock
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")
DB_PORT = os.getenv("DB_PORT")


engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SCHEME}')

In [2]:
def get_stock_history_columns(table_name):
  metadata = MetaData()
  table = Table(table_name, metadata, autoload_with=engine)
  db_columns = [column.name for column in table.columns]
  return db_columns

In [3]:
def get_price_data(at_date):
  df1 = stock.get_market_ohlcv(at_date, market="KOSPI")
  df2 = stock.get_market_ohlcv(at_date, market="KOSDAQ")

  result = pd.concat([df1, df2], axis=0)

  return result

In [4]:
def get_stock_id_by_short_code():
  Session = sessionmaker(bind=engine)
  session = Session()

  metadata = MetaData()
  stock_table = Table('STOCK', metadata, autoload_with=engine)

  # stock 테이블에서 short_code와 stock_id 조회
  query = session.query(stock_table.c.SHORT_CODE, stock_table.c.STOCK_ID).all()

  session.close()
  
  # short_code와 stock_id를 매핑한 딕셔너리 생성
  return {row.SHORT_CODE: row.STOCK_ID for row in query}


In [5]:
def convert_columns(data):
  data = data.reset_index()

  column_mapping = {
    '티커': 'SHORT_CODE',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
  }

  data.rename(columns=column_mapping, inplace=True)
  short_code_to_stock_id_map = get_stock_id_by_short_code()
  data['STOCK_ID'] = data['SHORT_CODE'].map(short_code_to_stock_id_map)

  data = data.dropna(subset=['STOCK_ID'])

  data['STOCK_ID'] = data['STOCK_ID'].astype(int)
  return data

In [6]:
def fetch_KOSPI_KOSDAQ_price_data():
  price = get_price_data(datetime.datetime.now().strftime("%Y%m%d"))
  price = convert_columns(price)
  return price[['STOCK_ID','SHORT_CODE','CLOSED_PRICE','OPEN_PRICE','HIGH_PRICE','LOW_PRICE']]

In [7]:
import numpy as np
import pandas as pd

def fetch_and_save_data(time):
  data = fetch_KOSPI_KOSDAQ_price_data()
  data['STOCK_HISTORY_ID'] = time
  data.to_sql(name='stock_history', con=engine, if_exists='append', index=False)
  print("Data saved at:", datetime.datetime.now())


In [8]:
import time
import datetime

def stock_market_schedule():
  while True:
    now = datetime.datetime.now()

    if now.hour >= 9 and (now.hour < 15 or (now.hour == 15 and now.minute <= 30)):
            
      if now.minute % 10 == 0 and now.second == 0:
        fetch_and_save_data(now.strftime("%Y%m%d%H%M%S"))        
      time.sleep(1)
# 스케줄 시작
stock_market_schedule()

C:\Users\student\AppData\Local\Temp\ipykernel_12748\4262470629.py:7: UserWarning: The provided table name 'STOCK_HISTORY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)


Data saved at: 2024-10-23 13:30:08.203943


C:\Users\student\AppData\Local\Temp\ipykernel_12748\4262470629.py:7: UserWarning: The provided table name 'STOCK_HISTORY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)


Data saved at: 2024-10-23 13:40:08.187821


C:\Users\student\AppData\Local\Temp\ipykernel_12748\4262470629.py:7: UserWarning: The provided table name 'STOCK_HISTORY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)


...

C:\Users\student\AppData\Local\Temp\ipykernel_12748\4262470629.py:7: UserWarning: The provided table name 'STOCK_HISTORY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)


Data saved at: 2024-10-23 14:10:07.893455


C:\Users\student\AppData\Local\Temp\ipykernel_12748\4262470629.py:7: UserWarning: The provided table name 'STOCK_HISTORY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)


Data saved at: 2024-10-23 14:20:08.241381


KeyboardInterrupt: 